In [ ]:
import vitaldb
import json
import os
import numpy as np
import time

save_path = ## set path here ##
track_names = ['SNUADC/PLETH']
target_freq = 100
max_duration = ## Set duration accordingly ##
max_samples = target_freq * max_duration

def save_data(case_id, samples):
    if os.path.exists(save_path):
        with open(save_path, "r") as file:
            try:
                data = json.load(file)
            except json.JSONDecodeError:
                data = {}
    else:
        data = {}

    data[str(case_id)] = samples.tolist()

    with open(save_path, "w") as file:
        json.dump(data, file, indent=2)

for case_id in range(741, 1000):
    print(f"Processing Case ID: {case_id}")

    try:
        vf = vitaldb.VitalFile(case_id, track_names)
        samples = vf.to_numpy(track_names, 1 / target_freq)

        if samples is None or samples.size == 0:
            print(f"No PPG data for Case {case_id}, skipping...")
            continue

        samples = samples[~np.isnan(samples)]

        if samples.size > max_samples:
            samples = samples[:max_samples]

        if samples.size == 0:
            print(f"Case {case_id} had only NaN values, skipping...")
            continue

        save_data(case_id, samples)
        print(f"Case {case_id} done. Saved {samples.size} samples.")

    except Exception as e:
        print(f"Error on Case {case_id}: {e}")

    time.sleep(0.1)

print("Done")